This shows how PyLabel can import data when given a YAML File and a dataset that has already had its images and annotations/labels folders split into train, test and val. This is our expected use case.

In [1]:
import torch
from IPython.display import Image  # for displaying images
import os, zipfile
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

import logging
logging.getLogger().setLevel(logging.CRITICAL)
!pip install pylabel > /dev/null

#!pip install pylabel

from pylabel import importer
from pylabel import *

from pathlib import PurePath

import yaml

In [2]:
#A random dataset that we found online.
!wget -O roadsign_splitdata.zip https://raw.githubusercontent.com/pylabel-project/datasets_models/main/roadsign_splitdata.zip
!unzip roadsign_splitdata.zip


--2021-11-30 01:46:42--  https://raw.githubusercontent.com/pylabel-project/datasets_models/main/roadsign_splitdata.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22454439 (21M) [application/zip]
Saving to: ‘roadsign_splitdata.zip’

roadsign_splitdata. 100%[===================>]  21.41M   137MB/s    in 0.2s    

2021-11-30 01:46:44 (137 MB/s) - ‘roadsign_splitdata.zip’ saved [22454439/22454439]

Archive:  roadsign_splitdata.zip
  inflating: annotations/test/road10.txt  
  inflating: annotations/test/road29.txt  
  inflating: annotations/test/road30.txt  
  inflating: annotations/test/road38.txt  
  inflating: annotations/test/road41.txt  
  inflating: annotations/train/road0.txt  
  inflating: annotations/train/road1.txt  
  inflating: annotations/train/

In [3]:
#An example annotation
!cat /content/annotations/train/road4.txt

1 0.1891 0.4325 0.2285 0.3200
1 0.5225 0.5425 0.1760 0.2750
1 0.7903 0.6050 0.1648 0.2650


In [4]:
#An example YAML file
!cat /content/road_sign_data.yaml

train: images/train/ 
val:  images/val/
test: images/test/

# number of classes
nc: 4

# class names
names: ["trafficlight","stop", "speedlimit","crosswalk"]

In [7]:
#Import the data from a YAML file and convert it to a dataset
data0 = importer.ImportYoloV5WithYaml(yaml_file="/content/road_sign_data.yaml", 
                     path_to_annotations=None, 
                     image_ext='png',
                     name_of_annotations_folder="annotations")

In [10]:
#An example of what the data looks like
data0.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,ann_bbox_xmax,ann_bbox_ymax,ann_bbox_width,ann_bbox_height,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,../../images/train,road23.png,NaN,0,266,400,3,NaN,216.00530,125.9800,241.99350,178.9800,25.9882,53.0000,1377.3746,NaN,NaN,NaN,NaN,NaN,1,stop,NaN,train,1
1,../../images/train,road33.png,NaN,1,267,400,3,NaN,23.00205,6.0200,177.99555,387.0200,154.9935,381.0000,59052.5235,NaN,NaN,NaN,NaN,NaN,1,stop,NaN,train,1
2,../../images/train,road8.png,NaN,2,400,300,3,NaN,91.00000,72.0150,129.00000,136.0050,38.0000,63.9900,2431.6200,NaN,NaN,NaN,NaN,NaN,1,stop,NaN,train,1
3,../../images/train,road8.png,NaN,2,400,300,3,NaN,245.02000,103.9950,274.02000,163.0050,29.0000,59.0100,1711.2900,NaN,NaN,NaN,NaN,NaN,1,stop,NaN,train,1
4,../../images/train,road16.png,NaN,3,400,248,3,NaN,225.00000,42.9908,241.00000,82.0012,16.0000,39.0104,624.1664,NaN,NaN,NaN,NaN,NaN,1,stop,NaN,train,1


In [8]:
#A sample of what the groupings look like
data0.df.groupby('split').count()

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,ann_bbox_xmax,ann_bbox_ymax,ann_bbox_width,ann_bbox_height,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,annotated
split,,,,,,,,,,,,,,,,,,,,,,,,
test,8,8,0,8,8,8,8,0,8,8,8,8,8,8,8,0,0,0,0,0,8,8,0,8
train,55,55,0,55,55,55,55,0,55,55,55,55,55,55,55,0,0,0,0,0,55,55,0,55
val,7,7,0,7,7,7,7,0,7,7,7,7,7,7,7,0,0,0,0,0,7,7,0,7
